# Advent of Code Day 4
https://adventofcode.com/2021/day/4

In [1]:
import os
from dataclasses import dataclass, field, InitVar
from typing import Optional, Tuple, List


In [2]:
@dataclass
class BingoSpace():
    value:int = field(default=0)
    marked:bool = field(default=False)

@dataclass
class BingoBoard():
    board: InitVar[list[list[int]]]
    spaces: list[BingoSpace] = field(default_factory=list)
    row_size: int = field(default=0, repr=False)
    col_size: int = field(default=0, repr=False)
    completed: bool = field(default=False, repr=False)

    def __post_init__(self, board: list[list[int]]):
        if len(board) == 0:
            raise ValueError("board is empty.")

        self.row_size = len(board)
        self.col_size = len(board[0])

        for row in board:
            if len(row) != self.col_size:
                raise ValueError("column lengths are mismatched.")

            for value in row:
                self.spaces.append(BingoSpace(value))

    def mark_space(self, value: int) -> None:
        for space in self.spaces:
            if space.value == value:
                space.marked = True
    
    def sum_marked(self) -> int:
        """
        Returns the sum of the marked BingoSpaces.
        """
        cum_sum: int = 0
        for space in self.spaces:
            if space.marked:
                cum_sum = cum_sum + space.value
        return cum_sum
    
    def sum_unmarked(self) -> int:
        """
        Returns the sum of the unmarked BingoSpaces.
        """
        cum_sum: int = 0
        for space in self.spaces:
            if not space.marked:
                cum_sum = cum_sum + space.value
        return cum_sum

    def space_index(self, row:int, col:int) -> int:
        return row * self.col_size + col

    def has_completed_row(self) -> bool:
        for r in range(self.row_size):
            row_complete: bool = True
            for c in range(self.col_size):
                idx = self.space_index(r, c)
                row_complete &= self.spaces[idx].marked
            if row_complete:
                return True
            row_complete = False
        return False 

    def has_completed_col(self) -> bool:
        for c in range(self.col_size):
            col_complete: bool = True
            for r in range(self.row_size):
                idx = self.space_index(r, c)
                col_complete &= self.spaces[idx].marked
            if col_complete:
                return True
        return False 

    def is_completed(self) -> bool:
        return self.has_completed_row() or self.has_completed_col()


def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_4'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_4'
) -> str:
    return os.path.join(data_directory, file_loc)

def read_input(input_file:str) -> Tuple[Optional[list[int]], list[BingoBoard]]:
    draw: Optional[list[int]] = None
    game_boards: list[BingoBoard] = []
    board = []

    with open(input_file) as f:
        for line in f:
            if line.rstrip() and draw is None:
                draw = [int(x) for x in line.rstrip().split(",")]
            
            elif not line.rstrip() and len(board) > 0:
                # commit a board
                game_boards.append(BingoBoard(board))
                board = []
            
            elif line.rstrip():
                row:list[int] = [int(x.strip()) for x in line.rstrip().split()]
                board.append(row)

        if len(board) > 0:  # capture last one
            game_boards.append(BingoBoard(board))

    return (draw, game_boards)


In [3]:
def winning_sum(draw, game_boards) -> int:
    for value in draw:
        for game_board in game_boards:
            game_board.mark_space(value)
            if game_board.is_completed():
                return game_board.sum_unmarked() * value

# test with test input data from the example.
draw, game_boards = read_input(test_input_location())
assert winning_sum(draw, game_boards) == 4512

### Find solution to part 1.
Finding the value of the board (sum unmarked spaces * drawn number) that wins first.

In [4]:
draw, game_boards = read_input(input_location())
winning_sum(draw, game_boards)

6592

### Find solution to part 2.
Finding the value of the board (sum unmarked spaces * drawn number) that wins last.

In [5]:
def last_winning_sum(draw, game_boards) -> int:
    completed_puzzles = 0
    
    for value in draw:
        for idx, game_board in enumerate(game_boards):
            if not game_board.completed:
                game_board.mark_space(value)

                if game_board.is_completed() and len(game_boards) - completed_puzzles == 1:
                    # print(f"Last winner is {idx} after value {value}")
                    return game_board.sum_unmarked() * value
                elif game_board.is_completed():
                    game_board.completed = True 
                    completed_puzzles = completed_puzzles + 1
                    # print(f"Removing {idx} after value {value} ({completed_puzzles} of {len(game_boards)})")

# test with test input data from the example.
draw, game_boards = read_input(test_input_location())
assert last_winning_sum(draw, game_boards) == 1924

In [6]:
draw, game_boards = read_input(input_location())
last_winning_sum(draw, game_boards)

31755